In [ ]:
import re
import pandas as pd

file = open('../input/20x20.txt','r')
string1 = file.readline()
s = re.findall('\d+ *\d*', string1)
data = s[0].split(" ")
COUNT_OF_MACHINES = int(data[0])
COUNT_OF_PARTS = int(data[1])
new_file = open('helper.csv', 'w+')
str1 = " ".join([str(i) for i in range(1,COUNT_OF_PARTS + 1)]) + "\n"
new_file.write(str1)

number = 1
for line in file:
    machine = line[:2]
    s2 = re.findall('\d+ ', line[2:])
    answer = ""
    for i in range(1,COUNT_OF_PARTS + 1):
        if str(i)+" " in s2:
            answer += "1 "
        else: 
            answer += "0 "
    new_file.write(str(number)+ " " + answer + "\n")
    number += 1

file.close()
new_file.close()

In [ ]:
problem = pd.read_csv('helper.csv', sep="\s+")

In [ ]:
import random
import numpy as np

matrix = [[[problem.iloc[j,i], [j,i]] for i in range(COUNT_OF_PARTS)] for j in range(COUNT_OF_MACHINES)]

matrix1 = [[problem.iloc[j,i] for i in range(COUNT_OF_PARTS)] for j in range(COUNT_OF_MACHINES)]
all_ones = np.sum(np.concatenate(matrix1))

### Deprecated functions for old way to create init solution:

In [ ]:
def print_matrix(some_matrix):
    for row in some_matrix:
        stri = "" 
        for point in row:
            if point[0] == 1:
                stri += str("# ")
            else:
                stri += str("_ ")
        print(stri)
        
def print_small_matrix(some_matrix):
    for row in some_matrix:
        stri = "" 
        for point in row:
            if point == 1:
                stri += str("# ")
            else:
                stri += str("_ ")
        print(stri)

In [ ]:
# print_matrix(matrix)

In [ ]:
def object_function_for_init(some_matrix, width):
    obj=0
    for i in np.arange(0,COUNT_OF_MACHINES,COUNT_OF_PARTS/COUNT_OF_MACHINES):
        for j in np.arange(0,COUNT_OF_PARTS, COUNT_OF_PARTS/COUNT_OF_MACHINES):
            i1 = int(round(i))
            j1 = int(round(j))
            if (i1 == j1 or abs(i1 - j1) <= width) and some_matrix[i1][j1][0] == 1:
                obj += 1
    return obj

In [ ]:
import copy
import itertools

def try_change_matrix(some_matrix, row1, row2, col1, col2):
    matr = copy.deepcopy(some_matrix)
    matr = change_columns(matr, col1, col2)
    matr = change_rows(matr, row1, row2)
    return matr
    
    
def change_columns(some_matrix, col1, col2):
    if abs(col1 - col2) != 1:
        return [some_matrix[i][:col1] + [some_matrix[i][col2]] + some_matrix[i][col1+1:col2] + [some_matrix[i][col1]] + some_matrix[i][col2+1:]\
                 for i in range(COUNT_OF_MACHINES)]
    else:
        return [some_matrix[i][:col1] + [some_matrix[i][col2]] + [some_matrix[i][col1]] + some_matrix[i][col2+1:]\
                 for i in range(COUNT_OF_MACHINES)]
    
def change_rows(some_matrix, row1, row2):
    if abs(row1 - row2) != 1:
        return some_matrix[:row1] + [some_matrix[row2]] + some_matrix[row1+1:row2] + [some_matrix[row1]]+ some_matrix[row2+1:] 
    else:
        return some_matrix[:row1] + [some_matrix[row2]] + [some_matrix[row1]]+ some_matrix[row2+1:] 

In [ ]:
def initial_solution(some_matrix, line_width):
    mat1 = copy.deepcopy(some_matrix)
    obj_init = object_function_for_init(some_matrix, line_width)
    for r1,r2 in itertools.combinations(range(COUNT_OF_MACHINES),2):
        for c1,c2 in itertools.combinations(range(COUNT_OF_PARTS),2):
            new_matrix = try_change_matrix(mat1,r1,r2,c1,c2)
            if object_function_for_init(new_matrix, line_width) > obj_init:
                obj_init = object_function_for_init(new_matrix, line_width)
                mat1 = copy.deepcopy(new_matrix)
    print_matrix(mat1)
    return mat1, obj_init

### Validation

In [ ]:
import numpy as np

def create_small_matrix(clust_machines, clust_parts):
    small_mat = [[0 for i in range(len(clust_parts))] for j in range(len(clust_machines))]
    row = 0
    for machine in clust_machines:
        column = 0      
        for part in clust_parts:
            small_mat[row][column] = problem.iloc[machine, part]
            column += 1
        row += 1
    return small_mat

def update_zeroes_and_ones(zeros, ones, matr):
    ones += np.sum(np.concatenate(matr))
    all_elem = 0
    for line in matr:
        for part in line:
            all_elem += 1
    zeros += all_elem - np.sum(np.concatenate(matr))
    return zeros, ones

def is_feasible(clust_machines, clust_parts):
    length = False
    all_m = False
    all_p = False
    if len(np.concatenate(clust_machines)) == COUNT_OF_MACHINES and len(np.concatenate(clust_parts)) == COUNT_OF_PARTS and len(clust_machines) == len(clust_parts):
        length = True
    if len(np.unique(np.concatenate(clust_machines))) == COUNT_OF_MACHINES:
        all_m = True
    if len(np.unique(np.concatenate(clust_parts))) == COUNT_OF_PARTS:
        all_p = True
    all_mp = all_m ^ all_p # 1^1 = 0
    return length ^ all_mp #1^0 = 1

In [ ]:
def object_function(clust_machines, clust_parts, some_matrix):
    zeros = 0
    ones = 0
    for i in range(len(clust_machines)):
        small_matrix = create_small_matrix(clust_machines[i], clust_parts[i])
        if small_matrix != []:
            zeros, ones = update_zeroes_and_ones(zeros, ones, small_matrix)
    return ones / (all_ones + zeros)

In [ ]:
import random
def make_cluster_with_random_size(clust_machines, clust_parts):
#     count_of_clusters = random.randint(5,10) 
    count_of_clusters = 5
    clusters_m = [[] for i in range(count_of_clusters)]
    clusters_p = [[] for i in range(count_of_clusters)]
    for i in range(count_of_clusters):
        clusters_m[i].append(i)
        clusters_p[i].append(i)
    
    for machine in clust_machines[count_of_clusters:]:
        clusterNum = random.randint(0,count_of_clusters-1)
        clusters_m[clusterNum].append(machine)
        
    for part in clust_parts[count_of_clusters:]:
        clusterNum = random.randint(0,count_of_clusters-1)
        clusters_p[clusterNum].append(part)
    
    return clusters_m, clusters_p

In [ ]:
import itertools

def swap_machines(clust_machines, clust_parts, matrixx):
    lens = [len(clust) for clust in clust_machines]
    obj = object_function(clust_machines, clust_parts, matrixx)
    all_machines = np.concatenate(clust_machines)
    for m1,m2 in itertools.combinations(range(len(all_machines)),2):
        if m2 == len(all_machines):
            new_machines = all_machines[:m1].tolist() + [all_machines[m2].tolist()] + all_machines[m1+1:m2].tolist()\
            + [all_machines[m1].tolist()] + all_machines[m2:].tolist()
        else:
            new_machines = all_machines[:m1].tolist() + [all_machines[m2].tolist()] + all_machines[m1+1:m2].tolist()\
            + [all_machines[m1].tolist()] +  all_machines[m2+1:].tolist()
        new_c_m = [[] for i in range(len(lens))]
        for i in range(len(lens)):
            new_c_m[i] = new_machines[:lens[i]]
            new_machines = new_machines[lens[i]:]
        new_obj = object_function(new_c_m, clust_parts, matrixx)
        if new_obj > obj and is_feasible(new_c_m, clust_parts):
            clust_machines = copy.deepcopy(new_c_m)
            obj = new_obj
    return clust_machines, obj

def swap_parts(clust_machines, clust_parts, matrixx):
    lens = [len(clust) for clust in clust_parts]
    obj = object_function(clust_machines, clust_parts, matrixx)
    all_parts = np.concatenate(clust_parts)
    for p1,p2 in itertools.combinations(range(len(all_parts)),2):
        if p2 == len(all_parts):
            new_parts = all_parts[:p1].tolist() + [all_parts[p2].tolist()] + all_parts[p1+1:p2].tolist()\
            + [all_parts[p1].tolist()] + all_parts[p2:].tolist()
        else:
            new_parts = all_parts[:p1].tolist() + [all_parts[p2].tolist()] + all_parts[p1+1:p2].tolist()\
            + [all_parts[p1].tolist()] +  all_parts[p2+1:].tolist()
        new_c_p = [[] for i in range(len(lens))]
        for i in range(len(lens)):
            new_c_p[i] = new_parts[:lens[i]]
            new_parts = new_parts[lens[i]:]
        new_obj = object_function(clust_machines, new_c_p, matrixx)
        if new_obj > obj and is_feasible(clust_machines, new_c_p):
            clust_parts = copy.deepcopy(new_c_p)
            obj = new_obj
    return clust_parts, obj



In [ ]:
def merge_clusters(clust_m, clust_p):
    return [clust_m[0] + clust_m[1], *[clust1 for clust1 in clust_m \
                                      if clust1 != clust_m[0] and clust1 != clust_m[1]]],\
           [clust_p[0] + clust_p[1], *[clust1 for clust1 in clust_p \
                                      if clust1 != clust_p[0] and clust1 != clust_p[1]]]
    
def split_clusters(clust_m, clust_p):
    return [clust_m[0][:round(len(clust_m[0])/2)],clust_m[0][round(len(clust_m[0])/2):] ,*[clust1 for clust1 in clust_m \
                               if clust1 != clust_m[0]]], \
           [clust_p[0][:round(len(clust_p[0])/2)],clust_p[0][round(len(clust_p[0])/2):], *[clust1 for clust1 in clust_p \
                               if clust1 != clust_p[0]]]

### Initial Solution

In [ ]:
m11 = [i for i in range(COUNT_OF_MACHINES)]
p11 = [i for i in range(COUNT_OF_PARTS)]

ObjF = 0

for j in range(10000):
    m1_c, p1_c = make_cluster_with_random_size(m11,p11)
    o1 = object_function(m1_c, p1_c, matrix)
    if o1 > ObjF:
        ObjF = o1       
# print(ObjF)

### Solution:

In [ ]:
shake = [merge_clusters, split_clusters]
oper = [swap_machines, swap_parts]
obj2 = ObjF
new_obj3 = 0
for i in range(100):
    for sh in shake:
        m1_c, p1_c = sh(m1_c, p1_c)
        while abs(new_obj3 - obj2) > 0.001:
            obj2 = new_obj3
            m1_c, new_obj2 = swap_machines(m1_c, p1_c, m)
            p1_c, new_obj3 = swap_parts(m1_c, p1_c, m)
            if new_obj3 > ObjF and is_feasible(m1_c, p1_c):
                ObjF = new_obj3
        random.shuffle(m1_c)
        random.shuffle(p1_c)

In [ ]:
def final_solution(clust_machines, clust_parts):
    str1 = [[i,-1] for i in range(COUNT_OF_MACHINES)]
    str2 = [[i,-1] for i in range(COUNT_OF_PARTS)]
    
    for i in range(len(clust_machines)):
        for m in clust_machines[i]:
            str1[m] = [m, i]    
    str_machines_answer=[]
    for i in str1:
        str_machines_answer.append(i[1])
    
    for i in range(len(clust_parts)):
        for p in clust_parts[i]:
            str2[p] = [p, i]    
    str_parts_answer=[]
    for i in str2:
        str_parts_answer.append(i[1])
        
    print(' '.join(str(i) for i in str_machines_answer))
    print(' '.join(str(i) for i in str_parts_answer))

In [ ]:
# final_solution(m1_c, p1_c)